In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import soundfile as sf

**Audio Filtering example:**  Below is a routine that will allow you to change the filtering ranges easily.

In [2]:
def plot_freq_respone(b, a, x_low=0, x_high=0.5, y_low=-60, y_high=0, freq_tag='', freq_scale=1):
    ## this computes the arma filter's frequency response from the diff. eq. coefficents
    w, H = signal.freqz(b, a, worN=2**16)
    
    plt.figure()
    nu =  ( w / ( 2 * np.pi ) ) 
    plt.plot(nu * freq_scale, 20 * np.log10(abs(H)), label='|H| (dB)')
    plt.legend()
    plt.grid(linestyle=':')
    xlims = np.asarray([x_low, x_high]) * freq_scale
    plt.xlim()
    plt.ylim([y_low, y_high])
    plt.xlabel(f'frequency {freq_tag}')
    plt.ylabel('filter gain (dB)')

def filter_and_plot_audio(f_cut_hz, filter_order, fname):
    ## read the wav file
    x, f_sample = sf.read(fname)

    nu_0 = f_cut_hz / f_sample
    b, a = signal.butter(filter_order, 2 * nu_0)
    plot_freq_respone(b, a, freq_tag='(kHz)', freq_scale=f_sample/1000)

    y = signal.lfilter(b, a, x)
    N_samples = len(x)
    t = np.arange(N_samples) / f_sample
    plt.figure()
    plt.plot(t, x, color = 'tab:gray', label='input') # linewidth=1,
    plt.plot(t, y, color = 'r', label='output')
    plt.legend()
    plt.grid(linestyle=':')
    plt.xlabel("time (sec)")
    plt.ylabel("signals")
    # plt.xlim([0, 100])
    plt.ylim([-1, 1])
    plt.show()
    #plt.savefig('toy.png', dpi=256)
    
    return y, f_sample


There are two audio files.  First, is a recording of me where I speak for a few seconds and then I (try to) whistle.  Let's filter out frequencies above 2 kHz and see what it sounds like.  This will look for a file called ``chugg_welcome.wav`` in a directory called ``data`` in this working directory.  If you are on windows, you will have to adjust the path definitions accordingly.

In [ ]:
f_cut = 2000
y_chirp, f_sample = filter_and_plot_audio(f_cut, 4, 'data/chugg_welcome.wav')
sf.write(f'data/chugg_welcome_{f_cut}.wav', y_chirp, f_sample)

Check out the spectrogram.  The top one is for the original file and the bottom one is for the filtered file.  The original was recorded using an apple wired headset, which has a smapling frequency of 44.1 kHz, so the maximum frequency that it can record is 22.05 kHz.  Note in the spectrograms that there are no high frequencies in the filter output -- you should be able to hear that too!  Note that a spectrogram is an advanced concept which shwos the frequency content as a function of time (we shall see why this is an advanced concept!)

![Spectrograms for Audio Example 1](img/chugg_welcome_spctrogram.png)


As an aside, here is a zoomed in picture at lower frequencies.  Note that there are specific patterns associated with the speech taht are visiable in a spectrogram.  Speech recognition systems are designed to spot the differences in these patterns for different sounds in our speech -- i.e., these spectrograms are typically the inputs to an automatic speech recognition engine. 

Also, see how the whstle is close to a pure tone at about 1.7 kHz? 

![Spectrograms for Audio Example 1](img/chugg_welcome_low_freqs.png)


OK, lets check out the second audio file which is `chirp.wav`.  This was generated using Audcity's built-in tools -- i.e., see the `Generate` menu.  A chip is a sine wave with frequency that increaces linearly over time.  For example, the frequency at time $t$ is $f(t) = \beta t$.  Then, the chirp signal is

$$x_c(t) = \cos( 2 \pi f(t) t) = \cos( 2 \pi \beta t^2) $$

When you listen to this, you will hear a tone that slowly increases frequency.  

Let's filter it to filter out frequencies above 4 kHz and plot the input and output.


In [ ]:
f_cut = 4000
y_chirp, f_sample = filter_and_plot_audio(f_cut, 4, 'data/chirp.wav')
sf.write(f'data/chirp_{f_cut}.wav', y_chirp, f_sample)


Notice in the above plot that the output is close to zero once the input frequency exceeds the filter cut-off frequency of 4 kHz. 

Let's take a look at the spectrograms from Audacity for this as well.  The top is the input chirp and the bottom is the output -- again note that the higher frequencies are attenuated by the filter.  

![Spectrograms for Audio Example 1](img/chip_spectrogram.png)
